# Load File

In [1]:
import pandas as pd
#Read the CSV file 'Churn_Modelling.csv' into a DataFrame named 'df' 
df = pd.read_csv('/Users/laiminyun/CIS 508/Group Project/Churn_Modelling.csv')

In [2]:
# Show the first few rows to understand the structure and content
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,$0.00,1,1,1,"$101,348.88",1
1,2,15647311,Hill,608,Spain,Female,41,1,"$83,807.86",1,0,1,"$112,542.58",0
2,3,15619304,Onio,502,France,Female,42,8,"$159,660.80",3,1,0,"$113,931.57",1
3,4,15701354,Boni,699,France,Female,39,1,$0.00,2,0,0,"$93,826.63",0
4,5,15737888,Mitchell,850,Spain,Female,43,2,"$125,510.82",1,1,1,"$79,084.10",0


In [3]:
df.shape

(10000, 14)

In [4]:
# Drop rows where the target variable or any of the specified columns have missing values
df = df.dropna()
df.shape

(10000, 14)

# Convert categorical variables to numerical variables ( 0 and 1)

In [5]:
# Geography column
unique_Geography = df['Geography'].unique()
df = pd.get_dummies(df, columns=['Geography'], prefix='Geography')

In [6]:
# EstimatedSalary Column
# Remove currency symbols and commas
df['EstimatedSalary'] = df['EstimatedSalary'].str.replace('[\$,]', '', regex=True)

# Convert the column to a numeric type
df['EstimatedSalary'] = pd.to_numeric(df['EstimatedSalary'])

In [7]:
# Balance Column
# Remove currency symbols and commas
df['Balance'] = df['Balance'].str.replace('[\$,]', '', regex=True)

# Convert the column to a numeric type
df['Balance'] = pd.to_numeric(df['Balance'])

In [8]:
df

,RowNumber,CustomerId,Surname,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,1,15634602,Hargrave,619,Female,42,2,0.00,1,1,1,101348.88,1,1,0,0
1,2,15647311,Hill,608,Female,41,1,83807.86,1,0,1,112542.58,0,0,0,1
2,3,15619304,Onio,502,Female,42,8,159660.80,3,1,0,113931.57,1,1,0,0
3,4,15701354,Boni,699,Female,39,1,0.00,2,0,0,93826.63,0,1,0,0
4,5,15737888,Mitchell,850,Female,43,2,125510.82,1,1,1,79084.10,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,Male,39,5,0.00,2,1,0,96270.64,0,1,0,0
9996,9997,15569892,Johnstone,516,Male,35,10,57369.61,1,1,1,101699.77,0,1,0,0
9997,9998,15584532,Liu,709,Female,36,7,0.00,1,0,1,42085.58,1,1,0,0
9998,9999,15682355,Sabbatini,772,Male,42,3,75075.31,2,1,0,92888.52,1,0,1,0


In [9]:
# Assume 'CustomerId', 'Surname' are irrelevant features
df.drop(['RowNumber','CustomerId', 'Surname','Gender'], axis=1, inplace=True)

In [10]:
# Separating the features and the target variable
X = df.drop('Exited', axis=1)  # Features
y = df['Exited']  # Target

# Modeling

In [11]:
!pip install scikit-learn


In [12]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import precision_recall_fscore_support
import pandas as pd

# Example: Generate a synthetic dataset
X, y = make_classification(n_samples=1000, n_features=20, n_classes=2, random_state=42)

# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize dictionaries to store 50th-epoch results for varying number of layers
final_results_layers = {}

# Loop over models with hidden layers from 3 to 10
for num_layers in range(3, 11):
    # Initialize the neural network classifier for the current model
    mlp_current = MLPClassifier(hidden_layer_sizes=(3,) * num_layers, max_iter=1, warm_start=True, solver='sgd')
    
    # Train for 50 epochs
    for epoch in range(50):
        # Fit model on training data
        mlp_current.fit(X_train, y_train)

    # Get predictions for the last epoch
    test_pred_current = mlp_current.predict(X_test)

    # Calculate precision, recall, and f1-score
    precision, recall, fscore, _ = precision_recall_fscore_support(y_test, test_pred_current)

    # Store results for the current model
    final_results_layers[num_layers] = {
        'Training_Accuracy_50th_Epoch': mlp_current.score(X_train, y_train),
        'Test_Accuracy_50th_Epoch': mlp_current.score(X_test, y_test),
        'Training_Loss_50th_Epoch': mlp_current.loss_,
        'Precision_Class_0': precision[0],
        'Precision_Class_1': precision[1],
        'Recall_Class_0': recall[0],
        'Recall_Class_1': recall[1],
        'F1_Score_Class_0': fscore[0],
        'F1_Score_Class_1': fscore[1]
    }

# Create a DataFrame to display the results in tabular form
results_df = pd.DataFrame(final_results_layers).T

results_df


/Users/laiminyun/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/laiminyun/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/laiminyun/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/laiminyun/anaconda3/lib/python3.11/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (1) reached and the optimization hasn't converged yet.
  warnings.warn(
/Users/laiminyun/anaconda3/l

,Training_Accuracy_50th_Epoch,Test_Accuracy_50th_Epoch,Training_Loss_50th_Epoch,Precision_Class_0,Precision_Class_1,Recall_Class_0,Recall_Class_1,F1_Score_Class_0,F1_Score_Class_1
3,0.532857,0.496667,0.693982,0.489865,1.000000,1.000000,0.025806,0.657596,0.050314
4,0.521429,0.486667,0.709975,0.482490,0.511628,0.855172,0.141935,0.616915,0.222222
5,0.592857,0.623333,0.671927,0.650943,0.608247,0.475862,0.761290,0.549801,0.676218
6,0.492857,0.516667,0.794167,0.000000,0.516667,0.000000,1.000000,0.000000,0.681319
7,0.507143,0.483333,0.730305,0.483333,0.000000,1.000000,0.000000,0.651685,0.000000
8,0.492857,0.516667,0.723529,0.000000,0.516667,0.000000,1.000000,0.000000,0.681319
9,0.507143,0.483333,0.753193,0.483333,0.000000,1.000000,0.000000,0.651685,0.000000
10,0.507143,0.483333,0.704460,0.483333,0.000000,1.000000,0.000000,0.651685,0.000000


# Summary